Begin by importing the libraries needed for data cleaning and preparation.

In [2]:
import numpy as np
import pandas as pd


#### Data was obtained from the MIMIC III database (https://mimic.physionet.org/).

The first step is to read in the first dataset of interest, ICUSTAYS, and have a first look at the columns and rows. <br> 


Remove patients who did not spend time in the ICU, since focus of this project is on ICU patients.

In [3]:
#read in ICUSTAYS dataset
icu_stays = pd.read_csv('ICUSTAYS.csv.gz', compression='gzip' )
print(icu_stays.shape)

(61532, 12)


#### Columns of interest in the ICU dataset: 
- INTIME
- HADM_ID

In [4]:
#convert INTIME to datetime format. The errors='coerce' argument allows for missing values
icu_stays.INTIME = pd.to_datetime(icu_stays.INTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

# check to see if there are any null dates
print('Number of missing intime:', icu_stays.INTIME.isnull().sum())


Number of missing intime: 0


In [5]:
#reduce the columns to only 'HADM_ID' and 'INTIME'
icu_stays = icu_stays[['HADM_ID', 'INTIME']]
icu_stays.head()

,HADM_ID,INTIME
0,110404,2198-02-14 23:27:38
1,106296,2170-11-05 11:05:29
2,188028,2128-06-24 15:05:20
3,173727,2120-08-07 23:12:42
4,164716,2186-12-25 21:08:04


In [6]:
#add column ICU to indicate whether or not patient had ICU stay (all values will be true)
icu_stays['ICU'] = True
icu_stays = icu_stays[['HADM_ID', 'ICU']]
icu_stays.head()


,HADM_ID,ICU
0,110404,True
1,106296,True
2,188028,True
3,173727,True
4,164716,True


In [7]:
#drop duplicate rows
print('There are ', len(icu_stays[icu_stays.duplicated(['HADM_ID'])]), 'duplicate hospital admissions.')


There are  3746 duplicate hospital admissions.


These admissions are patients who stayed more than once in the ICU on that admission.

In [8]:
icu_stays = icu_stays.drop_duplicates()

In [9]:
print('There are now', len(icu_stays[icu_stays.duplicated(['HADM_ID'])]), 'duplicate hospital admissions.')


There are now 0 duplicate hospital admissions.


In [10]:
icu_stays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57786 entries, 0 to 61531
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   HADM_ID  57786 non-null  int64
 1   ICU      57786 non-null  bool 
dtypes: bool(1), int64(1)
memory usage: 959.3 KB


In [11]:
admissions = pd.read_csv('ADMISSIONS.csv.gz', compression='gzip')
print(admissions.shape)


(58976, 19)


There are 58976 rows and 19 columns.

#### Columns of interest (or may be of interest) from the 'admissions' dataset include:
- HOSPITAL_EXPIRE_FLAG
- SUBJECT_ID
- HADM_ID
- ADMITTIME
- DEATHTIME
- ADMISSION_TYPE

In [12]:
# exploring the data to determine the datatypes, in particular, of the date columns.
admissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ROW_ID                58976 non-null  int64 
 1   SUBJECT_ID            58976 non-null  int64 
 2   HADM_ID               58976 non-null  int64 
 3   ADMITTIME             58976 non-null  object
 4   DISCHTIME             58976 non-null  object
 5   DEATHTIME             5854 non-null   object
 6   ADMISSION_TYPE        58976 non-null  object
 7   ADMISSION_LOCATION    58976 non-null  object
 8   DISCHARGE_LOCATION    58976 non-null  object
 9   INSURANCE             58976 non-null  object
 10  LANGUAGE              33644 non-null  object
 11  RELIGION              58518 non-null  object
 12  MARITAL_STATUS        48848 non-null  object
 13  ETHNICITY             58976 non-null  object
 14  EDREGTIME             30877 non-null  object
 15  EDOUTTIME             30877 non-null

Next, convert the dates to datetime format for processing.

In [13]:
#convert ADMITTIME and DEATHTIME to datetime format. The errors='coerce' argument allows for missing values
admissions.ADMITTIME = pd.to_datetime(admissions.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
admissions.DEATHTIME = pd.to_datetime(admissions.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
admissions.DISCHTIME = pd.to_datetime(admissions.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

# check to see if there are any null dates
print('Number of missing date admissions:', admissions.ADMITTIME.isnull().sum())
print('Number of missing DEATHTIME:', admissions.DEATHTIME.isnull().sum())
print('Number of missing DISCHTIME:', admissions.DISCHTIME.isnull().sum())


Number of missing date admissions: 0
Number of missing DEATHTIME: 53122
Number of missing DISCHTIME: 0


Now update the admissions dataset to include only HADM_ID that are in the icu_stays dataframe. Do this by completing a left merge on HADM_ID.

In [14]:
admissions.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'],
      dtype='object')

In [15]:
admissions = pd.merge(icu_stays[['HADM_ID']], admissions[['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA']],\
            on=('HADM_ID') , how='left')

In [16]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57786 entries, 0 to 57785
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   HADM_ID               57786 non-null  int64         
 1   ROW_ID                57786 non-null  int64         
 2   SUBJECT_ID            57786 non-null  int64         
 3   ADMITTIME             57786 non-null  datetime64[ns]
 4   DISCHTIME             57786 non-null  datetime64[ns]
 5   DEATHTIME             5813 non-null   datetime64[ns]
 6   ADMISSION_TYPE        57786 non-null  object        
 7   ADMISSION_LOCATION    57786 non-null  object        
 8   DISCHARGE_LOCATION    57786 non-null  object        
 9   INSURANCE             57786 non-null  object        
 10  LANGUAGE              32990 non-null  object        
 11  RELIGION              57330 non-null  object        
 12  MARITAL_STATUS        47796 non-null  object        
 13  ETHNICITY       

Next a new column ENDTIME will be created. This will indicate the when to end note collection for analysis - i.e., 24 hours after ADMITTIME or at time of death - whichever is sooner.


In [17]:
admissions['ADMIT+24'] = admissions['ADMITTIME'] + pd.DateOffset(days=1)
admissions['ENDTIME'] = admissions['ADMIT+24']

admissions.ENDTIME = np.where(admissions['DEATHTIME'] < admissions['ENDTIME'], admissions['DEATHTIME']+pd.DateOffset(hours=-2), admissions['ENDTIME'])
admissions.head()

,HADM_ID,ROW_ID,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,ADMIT+24,ENDTIME
0,110404,344,268,2198-02-11 13:40:00,2198-02-18 03:55:00,2198-02-18 03:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,CATHOLIC,SEPARATED,HISPANIC OR LATINO,2198-02-11 09:41:00,2198-02-11 15:18:00,DYSPNEA,1,1,2198-02-12 13:40:00,2198-02-12 13:40:00
1,106296,345,269,2170-11-05 11:04:00,2170-11-27 18:00:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,...,UNOBTAINABLE,SINGLE,WHITE,2170-11-05 07:22:00,2170-11-05 12:15:00,SEPSIS;PILONIDAL ABSCESS,0,1,2170-11-06 11:04:00,2170-11-06 11:04:00
2,188028,346,270,2128-06-23 18:26:00,2128-06-27 12:31:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,JEHOVAH'S WITNESS,MARRIED,UNKNOWN/NOT SPECIFIED,NaN,NaN,CAROTID STENOSIS\CAROTID ANGIOGRAM AND STENT,0,1,2128-06-24 18:26:00,2128-06-24 18:26:00
3,173727,347,271,2120-08-07 18:56:00,2120-08-20 16:00:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,...,NOT SPECIFIED,MARRIED,PATIENT DECLINED TO ANSWER,NaN,NaN,GALLSTONE PANCREATITIS,0,1,2120-08-08 18:56:00,2120-08-08 18:56:00
4,164716,348,272,2186-12-25 21:06:00,2187-01-02 14:57:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,...,UNOBTAINABLE,MARRIED,WHITE,NaN,NaN,PULMONARY EMBOLIS,0,1,2186-12-26 21:06:00,2186-12-26 21:06:00


In [18]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57786 entries, 0 to 57785
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   HADM_ID               57786 non-null  int64         
 1   ROW_ID                57786 non-null  int64         
 2   SUBJECT_ID            57786 non-null  int64         
 3   ADMITTIME             57786 non-null  datetime64[ns]
 4   DISCHTIME             57786 non-null  datetime64[ns]
 5   DEATHTIME             5813 non-null   datetime64[ns]
 6   ADMISSION_TYPE        57786 non-null  object        
 7   ADMISSION_LOCATION    57786 non-null  object        
 8   DISCHARGE_LOCATION    57786 non-null  object        
 9   INSURANCE             57786 non-null  object        
 10  LANGUAGE              32990 non-null  object        
 11  RELIGION              57330 non-null  object        
 12  MARITAL_STATUS        47796 non-null  object        
 13  ETHNICITY       

Twenty-four hours of notes need to be available in order to compare samples equally. Therefore, patients with a length of stay 24 hours or less are removed.

In [19]:
# first create a column that calculated length of stay
admissions['LOS_HOSP'] = admissions['DISCHTIME'] - admissions['ADMITTIME']
admissions.head(1)

,HADM_ID,ROW_ID,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,ADMIT+24,ENDTIME,LOS_HOSP
0,110404,344,268,2198-02-11 13:40:00,2198-02-18 03:55:00,2198-02-18 03:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,SEPARATED,HISPANIC OR LATINO,2198-02-11 09:41:00,2198-02-11 15:18:00,DYSPNEA,1,1,2198-02-12 13:40:00,2198-02-12 13:40:00,6 days 14:15:00


In [20]:
print('Number of hospitalizations more than 24 hours: ', len(admissions.loc[admissions['LOS_HOSP']>'24 hours']))
admissions = admissions.loc[admissions['LOS_HOSP']>'1 day']
admissions.shape


Number of hospitalizations more than 24 hours:  55727


(55727, 22)

In [22]:
# The next step is to add the target variable 'DEATH' which indicates whether or not a hospitalization resulted in death.
# positive death=1/negative death=0

#admissions['DEATH'] = admissions['DEATHTIME']>pd.Timestamp('00:00:00')
#admissions.DEATH.value_counts()
#print(admissions.DEATH.value_counts())
#print(admissions.DEATH.shape)

In [23]:
# I later realized that the column "HOSPITAL_EXPIRE_FLAG" is same as the "DEATH" column that was created.
# drop DEATH column
#admissions.drop('DEATH', axis=1, inplace=True)
#admissions.columns


In [24]:
admissions.shape

(55727, 22)

In [25]:
admissions.HOSPITAL_EXPIRE_FLAG.value_counts()

0    50856
1     4871
Name: HOSPITAL_EXPIRE_FLAG, dtype: int64

4871 out of 55727 hospital admissions resulted in death (8.7%). As should be expected, this dataset is imbalanced.

Now it's time to examine the caregiver notes. <br>
<br>
### Next: Read in the caregiver notes dataset and get an overview of its rows and features.


In [26]:
#Read in the caregiver notes dataset.

notes = pd.read_csv('NOTEEVENTS.csv.gz', compression='gzip')
notes.head(2)

C:\Users\caras\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...


In [27]:
notes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 174.8+ MB


ISERROR
A ‘1’ in the ISERROR column indicates that a physician has identified this note as an error. These rows will be removed.

In [28]:
notes.ISERROR.value_counts()

1.0    886
Name: ISERROR, dtype: int64

In [29]:
notes = notes.loc[notes['ISERROR']!=1]
notes.shape

(2082294, 11)

Columns of interest from the NOTES dataset:
- SUBJECT_ID
- HADM_ID
- CHARTDATE and CHARTTIME
- CATEGORY
- TEXT

In [30]:
# Viewing categories of caregiver notes to better understand the dataset.
notes.CATEGORY.value_counts()


Nursing/other        822497
Radiology            522279
Nursing              223182
ECG                  209051
Physician            141281
Discharge summary     59652
Echo                  45794
Respiratory           31701
Nutrition              9400
General                8236
Rehab Services         5408
Social Work            2661
Case Management         953
Pharmacy                101
Consult                  98
Name: CATEGORY, dtype: int64

In [31]:
notes.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2082294 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 190.6+ MB


CHARTDATE records the date at which the note was charted. CHARTDATE will always have a time value of 00:00:00.

CHARTTIME records the date and time at which the note was charted. If both CHARTDATE and CHARTTIME exist, then the date portions will be identical. All records have a CHARTDATE. A subset are missing CHARTTIME. More specifically, notes with a CATEGORY value of ‘Discharge Summary’, ‘ECG’, and ‘Echo’ never have a CHARTTIME, only CHARTDATE. Other categories almost always have both CHARTTIME and CHARTDATE, but there is a small amount of missing data for CHARTTIME (usually less than 0.5% of the total number of notes for that category).

In [32]:
#convert the chart dates and times into datetime format

notes.CHARTTIME = pd.to_datetime(notes.CHARTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
notes.CHARTDATE = pd.to_datetime(notes.CHARTDATE, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')


# check to see if there are any null dates
print('Number of missing CHARTTIME:', notes.CHARTTIME.isnull().sum())
print('Number of missing CHARTDATE:', notes.CHARTDATE.isnull().sum())



Number of missing CHARTTIME: 316566
Number of missing CHARTDATE: 0


Since there are so many missing CHARTTIME values, CHARTDATE will be used for processing and analysis (below). Before analyzing further, the dataframes will be merged.

In [33]:
admissions.columns

Index(['HADM_ID', 'ROW_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'ADMIT+24', 'ENDTIME',
       'LOS_HOSP'],
      dtype='object')

In [34]:
# Merging the 'admissions' and 'notes' together. A left merge is used so that all rows for hospital admissions are included
# and any caregiver notes that are not associated with a hospital admission are dropped.
df=pd.merge(admissions[['SUBJECT_ID', 'HADM_ID', 'LOS_HOSP','ADMITTIME', 'HOSPITAL_EXPIRE_FLAG', 'ADMISSION_TYPE', 'DEATHTIME','ADMIT+24', 'ENDTIME']],\
            notes[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CATEGORY','TEXT']], \
            on=('HADM_ID', 'SUBJECT_ID') , how='left', suffixes=('adm','note'))
print('There are ', len(df), 'rows and ',len(df.columns), 'columns.')


There are  1830890 rows and  12 columns.


In [35]:
list(df.columns)

['SUBJECT_ID',
 'HADM_ID',
 'LOS_HOSP',
 'ADMITTIME',
 'HOSPITAL_EXPIRE_FLAG',
 'ADMISSION_TYPE',
 'DEATHTIME',
 'ADMIT+24',
 'ENDTIME',
 'CHARTDATE',
 'CATEGORY',
 'TEXT']

Next update the dataframe to only include notes taken within 24 hours of admission, or up until 2 hours before time of death if patient expired in the first 24 hours.

In [36]:

df = df[df['CHARTDATE'] <= df['ENDTIME']]

In [37]:
df.shape

(428039, 12)

In order to combine all the text samples into one single TEXT row per admission, a new dataframe, 'text' is created. Next it will be merged back with the original dataframe with  CHARTTIME and CATEGORY columns dropped.

In [38]:
text = df[['HADM_ID', 'TEXT']]

In [39]:
grouped_HADM = text.groupby("HADM_ID")

grouped_text = grouped_HADM["TEXT"].agg(lambda column: "".join(column))

grouped_text = grouped_text.reset_index(name="TEXT")

print(grouped_text.head())


   HADM_ID                                               TEXT
0   100001  [**2117-9-11**] 11:12 AM\n CHEST (PA & LAT)   ...
1   100003  PATIENT/TEST INFORMATION:\nIndication: Left ve...
2   100006  Sinus tachycardia\nLeft axis deviation - anter...
3   100007  Sinus rhythm\nAtrial premature complex\nConsid...
4   100009  PATIENT/TEST INFORMATION:\nIndication: Abnorma...


In [40]:
grouped_text.shape

(53977, 2)

In [41]:
df.columns

Index(['SUBJECT_ID', 'HADM_ID', 'LOS_HOSP', 'ADMITTIME',
       'HOSPITAL_EXPIRE_FLAG', 'ADMISSION_TYPE', 'DEATHTIME', 'ADMIT+24',
       'ENDTIME', 'CHARTDATE', 'CATEGORY', 'TEXT'],
      dtype='object')

In [42]:
# Merging the 'grouped_text' and 'df' together. A left merge is used so that all rows for hospital admissions are included
# and any caregiver notes that are not associated with a hospital admission are dropped.
df2=pd.merge(grouped_text[['HADM_ID', 'TEXT']], df[['SUBJECT_ID', 'HADM_ID','LOS_HOSP', 'ADMITTIME', 'HOSPITAL_EXPIRE_FLAG',\
                                                    'ADMISSION_TYPE', 'DEATHTIME', 'ADMIT+24','ENDTIME']],\
             on=('HADM_ID') , how='left')
print('There are ', len(df2), 'rows and ',len(df2.columns), 'columns.')

There are  428039 rows and  10 columns.


In [43]:
df2 = df2.drop_duplicates()

In [44]:
print('There are now ', len(df2), 'rows and ', len(df2.columns), 'columns in df2.')

There are now  53977 rows and  10 columns in df2.


In [45]:
#check if any samples died before admission and remove them from the dataframe. Save as new dataframe.

df2.loc[df2['DEATHTIME'] < df2['ADMITTIME']]

,HADM_ID,TEXT,SUBJECT_ID,LOS_HOSP,ADMITTIME,HOSPITAL_EXPIRE_FLAG,ADMISSION_TYPE,DEATHTIME,ADMIT+24,ENDTIME
72966,117100,[**2119-6-8**] 9:52 PM\n CHEST (PORTABLE AP) ...,8697,2 days 22:51:00,2119-06-08 15:23:00,1,URGENT,2119-06-08 14:14:00,2119-06-09 15:23:00,2119-06-08 12:14:00


In [46]:
df2 = df2.drop(df2[df2['HADM_ID'] == 117100].index)
df2.shape

(53976, 10)

The data is now ready to begin processing!

In [47]:
#Saving the merged dataframe.
df2.to_csv('df2.csv', index=False)

For use in savings calculations (later), calculate the average length of hospital stay.

In [21]:
LOS_hosp_mean = admissions.LOS_HOSP.mean()
print('The average hospitalization length for patients who stayed in the ICU was ', LOS_hosp_mean)

The average hospitalization length for patients who stayed in the ICU was  10 days 13:56:24.582697


To also use in savings analysis, calculate the average number of days in the ICU per admission.

Read in ICU stays data so that we can determine average number of days in ICU

In [48]:
icu_stays = pd.read_csv('ICUSTAYS.csv.gz', compression='gzip' )
print(icu_stays.shape)
icu_stays.head(2)

(61532, 12)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788


#### Columns of interest in the ICU dataset: 
- INTIME
- OUTTIME
- LOS (length of stay)
- HADM_ID

In [49]:
#convert OUTTIME, and INTIME to datetime format. The errors='coerce' argument allows for missing values
icu_stays.OUTTIME = pd.to_datetime(icu_stays.OUTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
icu_stays.INTIME = pd.to_datetime(icu_stays.INTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

# check to see if there are any null dates
print('Number of missing outtime:', icu_stays.OUTTIME.isnull().sum())
print('Number of missing intime:', icu_stays.INTIME.isnull().sum())



Number of missing outtime: 10
Number of missing intime: 0


Remove the ICU stays with missing outtimes.

In [50]:
icu_stays = icu_stays.dropna(subset=['OUTTIME'])
print('Number of missing outtime:', icu_stays.OUTTIME.isnull().sum())


Number of missing outtime: 0


In [51]:
icu_stays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61522 entries, 0 to 61531
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ROW_ID          61522 non-null  int64         
 1   SUBJECT_ID      61522 non-null  int64         
 2   HADM_ID         61522 non-null  int64         
 3   ICUSTAY_ID      61522 non-null  int64         
 4   DBSOURCE        61522 non-null  object        
 5   FIRST_CAREUNIT  61522 non-null  object        
 6   LAST_CAREUNIT   61522 non-null  object        
 7   FIRST_WARDID    61522 non-null  int64         
 8   LAST_WARDID     61522 non-null  int64         
 9   INTIME          61522 non-null  datetime64[ns]
 10  OUTTIME         61522 non-null  datetime64[ns]
 11  LOS             61522 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(6), object(3)
memory usage: 6.1+ MB


In [52]:
ICU_LOS_mean = round(icu_stays.LOS.mean(), 2)
print('average length of ICU stay: ', ICU_LOS_mean, 'days')

average length of ICU stay:  4.92 days


In [53]:
df_los = icu_stays[['HADM_ID', 'LOS']]
df_los

,HADM_ID,LOS
0,110404,3.2490
1,106296,3.2788
2,188028,2.8939
3,173727,2.0600
4,164716,1.6202
...,...,...
61527,143774,2.1894
61528,123750,2.4942
61529,196881,0.9259
61530,118475,2.3346


In [54]:
df_los

,HADM_ID,LOS
0,110404,3.2490
1,106296,3.2788
2,188028,2.8939
3,173727,2.0600
4,164716,1.6202
...,...,...
61527,143774,2.1894
61528,123750,2.4942
61529,196881,0.9259
61530,118475,2.3346


In [55]:
print('There are ', len(df_los[df_los.duplicated(['HADM_ID'])]), 'duplicate hospital admissions.')


There are  3746 duplicate hospital admissions.


These admissions are patients who stayed more than once in the ICU on that admission. LOS_ICU values for these patients need to be combined for a total LOS_ICU for each admission.

In [56]:
df_los.loc[df_los['LOS'] < 0]


,HADM_ID,LOS


In [57]:
#add column, LOS_ICU_TOTAL 
#df['LOS_ICU'] = df['LOS_ICU'].dt.days
#df.LOS_ICU

#df.LOS_ICU_TOTAL = np.where(df[df.duplicated(['HADM_ID'])], df['LOS_ICU'].sum(), df['LOS_ICU'])

#df['is_dup'] = df[['lat', 'lon']].duplicated()
#df['dups'] = df.groupby(['lat','lon']).is_dup.transform(np.sum)
# df outputs:

df_los = df_los.groupby(['HADM_ID'], as_index=False)['LOS'].sum()


In [58]:
df_los.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57776 entries, 0 to 57775
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   HADM_ID  57776 non-null  int64  
 1   LOS      57776 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.3 MB


In [59]:
print('There are now', len(df_los[df_los.duplicated(['HADM_ID'])]), 'duplicate hospital admissions.')


There are now 0 duplicate hospital admissions.


In [60]:
ICU_LOS_mean_updated = round(df_los['LOS'].mean(), 2)

print('The average length of ICU stay was ', ICU_LOS_mean_updated, ' days.')

The average length of ICU stay was  5.24  days.


In [1]:
import pandas as pd
import numpy as np
df2 = pd.read_csv('df2.csv')

KeyboardInterrupt: 

In [ ]:
#confirming data types and that there are no null values
df2.info()

In [ ]:
#reorder the columns and remove those no longer needed.
df3 = df2[['HADM_ID', 'SUBJECT_ID', 'LOS_HOSP', 'HOSPITAL_EXPIRE_FLAG', 'TEXT']]

The last step before beginning data analysis is to split the data into training and test sets.

In [ ]:
# Preparing data further for ML model fitting, separating target variable from features

X = df3.drop(['HOSPITAL_EXPIRE_FLAG'], axis='columns') #feature columns
y = df3.HOSPITAL_EXPIRE_FLAG #target variable

# Split the data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, shuffle=True, random_state=42, stratify=y)

print('number of negative training samples: ', len(y_train[y_train==0]))
print('total number of negative training samples: ', len(y_train))
print('total number of test samples: ', len(y_test))
print('number of negative test samples: ', len(y_test[y_test==0]))

#Confirming target variable counts
y.value_counts()
print(len(y[y==1]), 'out of ', len(y), ' patients (', round(len(y[y==1])*100/len(y),1), '%) expired while in the ICU.') 

In [ ]:
# shuffle dataset in order to randomize all of the samples
df4 = df3.sample(n = len(df3), random_state = 42)
df4 = df4.reset_index(drop = True) #resetting index for the newly shuffled dataset


df_valid_test=df4.sample(frac=0.30,random_state=42) # Save 30% of the data as validation and testing data.
df_test = df_valid_test.sample(frac = 0.5, random_state = 42) #Of the validation & testing data, 50% is set aside for testing 
df_valid = df_valid_test.drop(df_test.index) #dropping the index for the validation data

df_train_all=df4.drop(df_valid_test.index) # remove the rows used for validation; We are left with the rest of the data which will be used for training


In [ ]:
df_train_all.head()

Now, working with the training data, split the data into 2 categories based on mortality.

In [ ]:
# split the training data into positive (death) and negative (no death)
positive = df_train_all.HOSPITAL_EXPIRE_FLAG == 1
df_train_pos = df_train_all.loc[positive]
df_train_neg = df_train_all.loc[~positive]
print('There are ', len(df_train_pos), 'positive samples and ', len(df_train_neg), 'negative samples.')
print('Percent positive samples: ', round(len(df_train_pos)*100/len(df_train_all),2),'%')

There is an imbalance in positive vs negative cases, which would be expected in this dataset. Therefore balancing the data is needed, so that the machine learning model does not always predict negative (no death). Sub-sampling the negative group is one method of doing this.

In [ ]:
#sub-sample negatives so there are an equal number of positive and negative samples.
df_train_neg = df_train_neg.sample(n=len(df_train_pos), random_state=42)

# merge the positive and negative samples into the final training set
df_train_final = pd.concat([df_train_pos, df_train_neg],axis = 0)

# shuffle the order of training samples 
df_train_final = df_train_final.sample(n = len(df_train_final), random_state = 42).reset_index(drop = True)

In [ ]:
len(df_train_final)


### Step 2: Preprocess text data using Bag of Words model.
#### What is Bag of Words?
<br>
Bag of Words is a method for extracting features from the text for use in machine learning algorithms. Basically, it breaks up a text into individual words, then counts how often each word occurs.

A vocabulary will be built using the training dataset. This vocabulary will later be used as a feature in the machine learning model as a basis to predict poitive or negative death.

In [ ]:
#viewing an example row of the text column
df_train_final.TEXT[1]

The text above shows that it needs some pre-processing (mainly removing the new line command ('\n'). One way to do this is to create a function to preprocess the text. This way the original data won't be modified.

In [ ]:
def preprocess_text(df4):
    # This function preprocesses the text by replacing new lines ('\n')  with a space.
    df4.TEXT = df4.TEXT.str.replace('\n',' ')
    return df4
# preprocess the text to deal with known issue
df_train_final = preprocess_text(df_train_final)
df_valid = preprocess_text(df_valid)
df_test = preprocess_text(df_test)

In [ ]:
df_train_final.TEXT[1]

Now import Python's Natural Language Toolkit (NLTK) and other necessary modules

In [ ]:
import nltk
from nltk import word_tokenize
import string # String module provides tools to manipulate strings
from nltk.corpus import stopwords

Next create a function to separate the text data into tokens (this is called tokenization). Tokens created here will be used to make a vocabulary (set of unique tokens) to be used as a feature for the model. All tokens or top K tokens can be used 

In [ ]:
number_list = string.digits
character_list = '\n'
character_list

In [ ]:
def text_tokenizer(text): # create a function that will tokenize the text, and also remove punctuation and numbers
    
    punc_list = string.punctuation #create list of punctuation marks
    number_list=string.digits #create list of numbers
    num_punc_list = number_list + punc_list +character_list #combine the lists together
    t = str.maketrans(dict.fromkeys(num_punc_list, " ")) # replace punctuation and numbers with spaces
    text = text.lower().translate(t) #lowercase all words
    tokens = word_tokenize(text) #tokenize the text 
    return tokens

Now that tokens can be created from the text, CountVectorizer will turn these tokens into number features to be used in the machine learning model. But first a list of stopwords will be created so that the machine can ignore these words when processing the tokens.

In [ ]:
nltk.download('stopwords')

stopwords = set(stopwords.words('english')) 


In [ ]:
import nltk
nltk.download('punkt')
# fit our vectorizer. 
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features = 3000, tokenizer = text_tokenizer, stop_words = stopwords)
vect.fit(df_train_final.TEXT.values)


In [ ]:
print(vect.vocabulary_)
print(type(vect))
print(vect.fixed_vocabulary_)

In [ ]:
df_train_final.head()


We can get an idea of the frequency of words for positive vs negative mortality. This section was based on code from https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-2-333514854913

In [ ]:
#Next visualize find the most frequent words.

import matplotlib.pyplot as plt

neg_doc_matrix = vect.transform(df_train_final[df_train_final.HOSPITAL_EXPIRE_FLAG == 0].TEXT)
pos_doc_matrix = vect.transform(df_train_final[df_train_final.HOSPITAL_EXPIRE_FLAG == 1].TEXT)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))

term_freq_df = pd.DataFrame([neg,pos],columns=vect.get_feature_names()).transpose()
term_freq_df.columns = ['negative', 'positive']
term_freq_df['total'] = term_freq_df['negative'] + term_freq_df['positive']
term_freq_df.sort_values(by='total', ascending=False).iloc[:20]


In [ ]:
#Create a series from the sparse matrix
d = pd.Series(term_freq_df.total, 
              index = term_freq_df.index).sort_values(ascending=False)

#Visualize the 100 most frequent words in the text
ax = d[:50].plot(kind='bar', figsize=(10,6), width=.8, fontsize=14, rot=90,color = 'b')
ax.title.set_size(18)
plt.title('Most Frequent Words in Caregiver Notes')
plt.ylabel('count')
plt.show()
ax = d[50:100].plot(kind='bar', figsize=(10,6), width=.8, fontsize=14, rot=90,color = 'b')
ax.title.set_size(18)
plt.ylabel('count')
plt.show()

In [ ]:
#with np.printoptions(threshold=np.inf):
 #   print(neg_doc_matrix)

In [ ]:
stopwords_updated = list(stopwords) + ['pt', 'left', 'right', 'name','patient', 'p','c']


Add the list of stop words as an argument for CountVectorizer

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features = 3000, 
                       ngram_range = (1,2)
                       tokenizer = text_tokenizer, 
                       stop_words = stopwords_updated)
# this will take a while
vect.fit(df_train_final.TEXT.values)


In [ ]:
#Again visualize find the most frequent words.

import matplotlib.pyplot as plt

neg_doc_matrix = vect.transform(df_train_final[df_train_final.HOSPITAL_EXPIRE_FLAG == 0].TEXT)
pos_doc_matrix = vect.transform(df_train_final[df_train_final.HOSPITAL_EXPIRE_FLAG == 1].TEXT)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))

term_freq_df = pd.DataFrame([neg,pos],columns=vect.get_feature_names()).transpose()
term_freq_df.columns = ['negative', 'positive']
term_freq_df['total'] = term_freq_df['negative'] + term_freq_df['positive']
term_freq_df.sort_values(by='total', ascending=False).iloc[:20]


In [ ]:
#Again Create a series from the sparse matrix
d = pd.Series(term_freq_df.total, 
              index = term_freq_df.index).sort_values(ascending=False)

#Visualize the 100 most frequent words in the text
ax = d[:50].plot(kind='bar', figsize=(10,6), width=.8, fontsize=14, rot=90,color = 'b')
ax.title.set_size(18)
plt.title('Most Frequent Words in Caregiver Notes')
plt.ylabel('count')
plt.show()
ax = d[50:100].plot(kind='bar', figsize=(10,6), width=.8, fontsize=14, rot=90,color = 'b')
ax.title.set_size(18)
plt.ylabel('count')
plt.show() 

Transform the text samples into vectors

In [ ]:
X_train_vect = vect.transform(df_train_final.TEXT.values)
X_valid_vect = vect.transform(df_valid.TEXT.values)
X_test_vect = vect.transform(df_test.TEXT.values)

In [ ]:
X_train_vect.shape

Get labels (target variable)

In [ ]:
y_train = df_train_final.HOSPITAL_EXPIRE_FLAG
y_valid = df_valid.HOSPITAL_EXPIRE_FLAG
y_test = df_test.HOSPITAL_EXPIRE_FLAG

Step 3: Build a simple predictive model

In [ ]:
# logistic regression
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(random_state = 42, solver='lbfgs')
clf.fit(X_train_vect, y_train)

In [ ]:
#import other models for evaluation
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline

rf = RandomForestClassifier(random_state=42, n_estimators=100)
svm = SVC(random_state=42, gamma='auto', probability=True)
gnb = GaussianNB()


rf.fit(X_train_vect, y_train)

svm.fit(X_train_vect, y_train)

gnb.fit(X_train_vect.toarray(), y_train)



Calculate probability of death.

In [ ]:
model = clf
y_train_preds = model.predict_proba(X_train_vect)[:,1]
y_valid_preds = model.predict_proba(X_valid_vect)[:,1]

In [ ]:
#Print mortality predictions for the first 10 samples in the training set
print(y_train[:10].values)
print(y_train_preds[:10])


In [ ]:
model = gnb
y_train_preds_gnb = model.predict_proba(X_train_vect.toarray())[:,1]
y_valid_preds_gnb = model.predict_proba(X_valid_vect.toarray())[:,1]

In [ ]:
model=rf
y_train_preds_rf = model.predict_proba(X_train_vect)[:,1]
y_valid_preds_rf = model.predict_proba(X_valid_vect.toarray())[:,1]

In [ ]:
model = svm
y_train_preds_svm = model.predict_proba(X_train_vect)[:,1]
y_valid_preds_svm = model.predict_proba(X_valid_vect.toarray())[:,1]

In [ ]:
#Print mortality predictions for the first 10 samples in the training set
print(y_train[:10].values)
print(y_train_preds[:10])


Calculate performance metrics

In [ ]:
#Create functions to calculate performanece metrics
def calc_accuracy(y_actual, y_pred, thresh):
    # this function calculates the accuracy with probability threshold at thresh
    return (sum((y_pred > thresh) & (y_actual == 1))+sum((y_pred < thresh) & (y_actual == 0))) /len(y_actual)

def calc_recall(y_actual, y_pred, thresh):
    # calculates the recall
    return sum((y_pred > thresh) & (y_actual == 1)) /sum(y_actual)

def calc_precision(y_actual, y_pred, thresh):
    # calculates the precision
    return sum((y_pred > thresh) & (y_actual == 1)) /sum(y_pred > thresh)

def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

def calc_prevalence(y_actual):
    # calculates prevalence
    return sum((y_actual == 1)) /len(y_actual)

In [ ]:
#LOGISTIC REGRESSION
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_preds)
fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_valid, y_valid_preds)


thresh = 0.5

auc_train = roc_auc_score(y_train, y_train_preds)
auc_valid = roc_auc_score(y_valid, y_valid_preds)

print("\nLogistic Regression Performance Metrics\n")
print('AUC:')
print('Train:%.3f'%auc_train)
print('Valid:%.3f'%auc_valid)

print('\nAccuracy:')
print('Train:%.3f'%calc_accuracy(y_train, y_train_preds, thresh))
print('Valid:%.3f'%calc_accuracy(y_valid, y_valid_preds, thresh))

print('\nRecall:')
print('Train:%.3f'%calc_recall(y_train, y_train_preds, thresh))
print('Valid:%.3f'%calc_recall(y_valid, y_valid_preds, thresh))

print('\nPrecision:')
print('Train:%.3f'%calc_precision(y_train, y_train_preds, thresh))
print('Valid:%.3f'%calc_precision(y_valid, y_valid_preds, thresh))

print('\nSpecificity')
print('Train:%.3f'%calc_specificity(y_train, y_train_preds, thresh))
print('Valid:%.3f'%calc_specificity(y_valid, y_valid_preds, thresh))

print('\nPrevalence')
print('Train:%.3f'%calc_prevalence(y_train))
print('Valid:%.3f'%calc_prevalence(y_valid))


plt.plot(fpr_train, tpr_train,'r-', label = 'Train AUC: %.2f'%auc_train)
plt.plot(fpr_valid, tpr_valid,'b-',label = 'Valid AUC: %.2f'%auc_valid)
plt.plot([0,1],[0,1],'-k')
plt.title('Logistic Regression AUC-ROC')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

In [ ]:
#GNB
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_preds_gnb)
fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_valid, y_valid_preds_gnb)


thresh = 0.5

auc_train = roc_auc_score(y_train, y_train_preds_gnb)
auc_valid = roc_auc_score(y_valid, y_valid_preds_gnb)
print("\nNaive Bayes Performance Metrics\n")
print('Train AUC:%.3f'%auc_train)
print('Valid AUC:%.3f'%auc_valid)

print('Train accuracy:%.3f'%calc_accuracy(y_train, y_train_preds_gnb, thresh))
print('Valid accuracy:%.3f'%calc_accuracy(y_valid, y_valid_preds_gnb, thresh))


print('Train recall:%.3f'%calc_recall(y_train, y_train_preds_gnb, thresh))
print('Valid recall:%.3f'%calc_recall(y_valid, y_valid_preds_gnb, thresh))

print('Train precision:%.3f'%calc_precision(y_train, y_train_preds_gnb, thresh))
print('Valid precision:%.3f'%calc_precision(y_valid, y_valid_preds_gnb, thresh))

print('Train specificity:%.3f'%calc_specificity(y_train, y_train_preds_gnb, thresh))
print('Valid specificity:%.3f'%calc_specificity(y_valid, y_valid_preds_gnb, thresh))

print('Train prevalence:%.3f'%calc_prevalence(y_train))
print('Valid prevalence:%.3f'%calc_prevalence(y_valid))


plt.plot(fpr_train, tpr_train,'r-', label = 'Train AUC: %.2f'%auc_train)
plt.plot(fpr_valid, tpr_valid,'b-',label = 'Valid AUC: %.2f'%auc_valid)
plt.plot([0,1],[0,1],'-k')
plt.title('Naive Bayes AUC-ROC')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

In [ ]:
#RF
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_preds_rf)
fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_valid, y_valid_preds_rf)


thresh = 0.5

auc_train = roc_auc_score(y_train, y_train_preds_rf)
auc_valid = roc_auc_score(y_valid, y_valid_preds_rf)
print("\nRandom Forest Performance Metrics\n")
print('Train AUC:%.3f'%auc_train)
print('Valid AUC:%.3f'%auc_valid)

print('Train accuracy:%.3f'%calc_accuracy(y_train, y_train_preds_rf, thresh))
print('Valid accuracy:%.3f'%calc_accuracy(y_valid, y_valid_preds_rf, thresh))


print('Train recall:%.3f'%calc_recall(y_train, y_train_preds_rf, thresh))
print('Valid recall:%.3f'%calc_recall(y_valid, y_valid_preds_rf, thresh))

print('Train precision:%.3f'%calc_precision(y_train, y_train_preds_rf, thresh))
print('Valid precision:%.3f'%calc_precision(y_valid, y_valid_preds_rf, thresh))

print('Train specificity:%.3f'%calc_specificity(y_train, y_train_preds_rf, thresh))
print('Valid specificity:%.3f'%calc_specificity(y_valid, y_valid_preds_rf, thresh))

print('Train prevalence:%.3f'%calc_prevalence(y_train))
print('Valid prevalence:%.3f'%calc_prevalence(y_valid))


plt.plot(fpr_train, tpr_train,'r-', label = 'Train AUC: %.2f'%auc_train)
plt.plot(fpr_valid, tpr_valid,'b-',label = 'Valid AUC: %.2f'%auc_valid)
plt.plot([0,1],[0,1],'-k')
plt.title('Random Forest AUC-ROC')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

In [ ]:
#svm
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_preds_svm)
fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_valid, y_valid_preds_svm)


thresh = 0.5

auc_train = roc_auc_score(y_train, y_train_preds_svm)
auc_valid = roc_auc_score(y_valid, y_valid_preds_svm)
print("\nSVM Performance Metrics\n")
print('Train AUC:%.3f'%auc_train)
print('Valid AUC:%.3f'%auc_valid)

print('Train accuracy:%.3f'%calc_accuracy(y_train, y_train_preds_svm, thresh))
print('Valid accuracy:%.3f'%calc_accuracy(y_valid, y_valid_preds_svm, thresh))


print('Train recall:%.3f'%calc_recall(y_train, y_train_preds_svm, thresh))
print('Valid recall:%.3f'%calc_recall(y_valid, y_valid_preds_svm, thresh))

print('Train precision:%.3f'%calc_precision(y_train, y_train_preds_svm, thresh))
print('Valid precision:%.3f'%calc_precision(y_valid, y_valid_preds_svm, thresh))

print('Train specificity:%.3f'%calc_specificity(y_train, y_train_preds_svm, thresh))
print('Valid specificity:%.3f'%calc_specificity(y_valid, y_valid_preds_svm, thresh))

print('Train prevalence:%.3f'%calc_prevalence(y_train))
print('Valid prevalence:%.3f'%calc_prevalence(y_valid))


plt.plot(fpr_train, tpr_train,'r-', label = 'Train AUC: %.2f'%auc_train)
plt.plot(fpr_valid, tpr_valid,'b-',label = 'Valid AUC: %.2f'%auc_valid)
plt.plot([0,1],[0,1],'-k')
plt.title('SVM AUC-ROC')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

### Since the performance metrics results and AUC-ROC fared better with Logistic Regression, that will be the model of choice for the remainder of this project.
#### The remainder of this analysis will focus on feature engineering and hyperparameter optimization in order to create a stronger algorithm to predict mortality. 

### Feature Importance

In [ ]:
#First find the features that the classifier is using to make decisions. The features with the highest coefficients predict death and the lowest coefficents predict no death.

def get_most_important_features(vectorizer, model, n=5):
    index_to_word = {v:k for k,v in vectorizer.vocabulary_.items()}
    
    # loop for each class
    classes ={}
    for class_index in range(model.coef_.shape[0]):
        word_importances = [(el, index_to_word[i]) for i,el in enumerate(model.coef_[class_index])]
        sorted_coeff = sorted(word_importances, key = lambda x : x[0], reverse=True)
        tops = sorted(sorted_coeff[:n], key = lambda x : x[0])
        bottom = sorted_coeff[-n:]
        classes[class_index] = {
            'tops':tops,
            'bottom':bottom
        }
    return classes

importance = get_most_important_features(vect, clf, 50)

In [ ]:
#Next, plot the most important features

def plot_important_words(top_scores, top_words, bottom_scores, bottom_words, name):
    y_pos = np.arange(len(top_words))
    top_pairs = [(a,b) for a,b in zip(top_words, top_scores)]
    top_pairs = sorted(top_pairs, key=lambda x: x[1])
    
    bottom_pairs = [(a,b) for a,b in zip(bottom_words, bottom_scores)]
    bottom_pairs = sorted(bottom_pairs, key=lambda x: x[1], reverse=True)
    
    top_words = [a[0] for a in top_pairs]
    top_scores = [a[1] for a in top_pairs]
    
    bottom_words = [a[0] for a in bottom_pairs]
    bottom_scores = [a[1] for a in bottom_pairs]
    
    fig = plt.figure(figsize=(10, 15))  

    plt.subplot(121)
    plt.barh(y_pos,bottom_scores, align='center', alpha=0.5)
    plt.title('Negative', fontsize=20)
    plt.yticks(y_pos, bottom_words, fontsize=14)
    plt.suptitle('Key words', fontsize=16)
    plt.xlabel('Importance', fontsize=20)
    
    plt.subplot(122)
    plt.barh(y_pos,top_scores, align='center', alpha=0.5)
    plt.title('Positive', fontsize=20)
    plt.yticks(y_pos, top_words, fontsize=14)
    plt.suptitle(name, fontsize=16)
    plt.xlabel('Importance', fontsize=20)
    
    plt.subplots_adjust(wspace=0.8)
    plt.show()

    
  
top_scores = [a[0] for a in importance[0]['tops']]
top_words = [a[1] for a in importance[0]['tops']]
bottom_scores = [a[0] for a in importance[0]['bottom']]
bottom_words = [a[1] for a in importance[0]['bottom']]

plot_important_words(top_scores, top_words, bottom_scores, bottom_words, "Most important words")


### Hyperparameter tuning

In [ ]:
#Begin by plotting a learning curve. This will help to assure that we have enough samples to make good predictions.  

import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
        """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Number of Training examples")
    plt.ylabel("AUC")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring = 'roc_auc')
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="b")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="b",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:

title = "Learning Curves (Logistic Regression)"
# Cross validation with 5 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
estimator = LogisticRegression( C = 0.0001, penalty = 'l2')#
plot_learning_curve(estimator, title, X_train_vect, y_train, ylim=(0.2, 1.01), cv=cv, n_jobs=4)

plt.show()


The AUC of the training set is high, and the CV score is also high, and begins to approach the Training score curve as number of samples increases. This means that the  model is learning well with the number of samples provided; more data would not necessarily make the model better. Some overfitting is indicated since there is a gap between the training score and cross validation score.

In [ ]:
len(clf.coef_.values())

In [ ]:
#Look at the most important words for the model.

feature_importances = pd.DataFrame(clf.coef_[0],
                                   index = vect.get_feature_names(),
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)

num=25
ylocs = np.arange(num)
# get the feature importance for top num and sort in reverse order
values_to_plot = feature_importances.iloc[:num].values.ravel()[::-1]
feature_labels = list(feature_importances.iloc[:num].index)[::-1]

plt.figure(num=None, figsize=(8, 15), dpi=80, facecolor='w', edgecolor='k');
plt.barh(ylocs, values_to_plot, align = 'center')
plt.ylabel('Features')
plt.xlabel('Importance Score')
plt.title('Positive Feature Importance Score - Logistic Regression')
plt.yticks(ylocs, feature_labels)
plt.show()

values_to_plot = feature_importances.iloc[-num:].values.ravel()
feature_labels = list(feature_importances.iloc[-num:].index)

plt.figure(num=None, figsize=(8, 15), dpi=80, facecolor='w', edgecolor='k');
plt.barh(ylocs, values_to_plot, align = 'center')
plt.ylabel('Features')
plt.xlabel('Importance Score')
plt.title('Negative Feature Importance Score - Logistic Regression')
plt.yticks(ylocs, feature_labels)
plt.show()




In [ ]:
# Find best value of C


from sklearn.linear_model import LogisticRegression

Cs = [0.00001, 0.0001, 0.001, 0.005, 0.01, 0.05, 0.1]
train_aucs = np.zeros(len(Cs))
valid_aucs = np.zeros(len(Cs))

for ii in range(len(Cs)):
    C = Cs[ii]
    print('\n C:', C)
    
    # logistic regression
    
    clf=LogisticRegression(C = C, penalty = 'l2', random_state = 42)
    clf.fit(X_train_vect, y_train)

    model = clf
    y_train_preds = model.predict_proba(X_train_vect)[:,1]
    y_valid_preds = model.predict_proba(X_valid_vect)[:,1]

    auc_train = roc_auc_score(y_train, y_train_preds)
    auc_valid = roc_auc_score(y_valid, y_valid_preds)
    print('Train AUC:%.3f'%auc_train)
    print('Valid AUC:%.3f'%auc_valid)
    train_aucs[ii] = auc_train
    valid_aucs[ii] = auc_valid


plt.plot(Cs, train_aucs,'bo-', label ='Train')
plt.plot(Cs, valid_aucs, 'ro-', label='Valid')
plt.legend()
plt.xlabel('Logistic Regression - C')
plt.ylabel('AUC')
plt.show()







As C gets larger, overfitting quickly occurs as shown by the gap between training and validation. The best values for C are 0.0001 and 0.001, based on the individual ROC-AUC and difference between  training and validation ROC-AUC scores at those values of C.

In [ ]:
#Countvectorizer is currently using 3000 maximum tokens. Next examine the effect of Max_features on performance.





num_features = [100,300,1000,3000,10000,30000]
train_aucs = np.zeros(len(num_features))
valid_aucs = np.zeros(len(num_features))

for ii in range(len(num_features)):
    num = num_features[ii]
    print('\nnumber of features:', num)
    vect = CountVectorizer(lowercase = True, max_features = num, 
                           tokenizer = text_tokenizer,stop_words =stopwords_updated)

    # This could take a while
    vect.fit(df_train_final.TEXT.values)

    X_train_vect = vect.transform(df_train_final.TEXT.values)
    X_valid_vect = vect.transform(df_valid.TEXT.values)
    y_train = df_train_final.HOSPITAL_EXPIRE_FLAG
    y_valid = df_valid.HOSPITAL_EXPIRE_FLAG
    
    clf=LogisticRegression(C = 0.0001, penalty = 'l2', random_state = 42)
    clf.fit(X_train_vect, y_train)

    model = clf
    y_train_preds = model.predict_proba(X_train_vect)[:,1]
    y_valid_preds = model.predict_proba(X_valid_vect)[:,1]

    auc_train = roc_auc_score(y_train, y_train_preds)
    auc_valid = roc_auc_score(y_valid, y_valid_preds)
    print('Train AUC: %.3f'%auc_train)
    print('Valid AUC:%.3f'%auc_valid)
    train_aucs[ii] = auc_train
    valid_aucs[ii] = auc_valid

plt.plot(num_features, train_aucs,'bo-', label ='Train')
plt.plot(num_features, valid_aucs, 'ro-', label='Valid')
plt.legend()
plt.xlabel('Number of features')
plt.ylabel('AUC')
plt.show()

As max_features gets larger, some overfitting quickly occurs. AUC begins leveling out for both Training and Validation at about 3000-4000 words. Therefore, the current setting for max_features is appropriate.

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

clf_param_grid = {'penalty': ['l1','l2'],
                  'C': [0.001, 0.01, 0.1, 1, 2],
                  'class_weight': [{1:0.5, 0:0.5}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}, {1:0.8, 0:0.2}, {1:0.9, 0:0.1}]
                   }
clf = GridSearchCV(
    estimator=clf,
    param_grid=clf_param_grid,               
    n_jobs=-1,
    scoring='f1',
    cv=StratifiedKFold(n_splits=5,shuffle=True))
logistic_best_model = clf.fit(X_train_vect, y_train)

print('Best Logistic Score (F1): ', logistic_best_model.best_score_)
print('Best Logistic Parameters: ', logistic_best_model.best_params_)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix 

#Re-running predictions and see classification report on this grid object
clf_best_model = logistic_best_model.predict(X_test_vect) 

print('Logistic Classification Report\n\n', classification_report(y_test, clf_best_model))
print('Confusion Matrix \n\n', confusion_matrix(y_test, clf_best_model))

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_best=LogisticRegression(C = 0.001, penalty = 'l2',class_weight={1: 0.6, 0: 0.4}, random_state = 42, n_jobs=-1)
clf_best.fit(X_train_vect, y_train)

model = clf_best


In [ ]:
y_train_preds = model.predict_proba(X_train_vect)[:,1]
y_valid_preds = model.predict_proba(X_valid_vect)[:,1]
y_test_preds = model.predict_proba(X_test_vect)[:,1]


In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_preds)
fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_valid, y_valid_preds)
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_test_preds)


thresh = 0.5

auc_train = roc_auc_score(y_train, y_train_preds)
auc_valid = roc_auc_score(y_valid, y_valid_preds)
auc_test = roc_auc_score(y_test, y_test_preds)

print("\nBest Model Performance Metrics\n")
print('\nAUC:')
print('Train:%.3f'%auc_train)
print('Valid:%.3f'%auc_valid)
print('Test:%.3f'%auc_test)

print('\nAccuracy')
print('Train:%.3f'%calc_accuracy(y_train, y_train_preds, thresh))
print('Valid:%.3f'%calc_accuracy(y_valid, y_valid_preds, thresh))
print('Test:%.3f'%calc_accuracy(y_test, y_test_preds, thresh))

print('\nRecall')
print('Train:%.3f'%calc_recall(y_train, y_train_preds, thresh))
print('Valid:%.3f'%calc_recall(y_valid, y_valid_preds, thresh))
print('Test:%.3f'%calc_recall(y_test, y_test_preds, thresh))

print('\nPrecision')
print('Train:%.3f'%calc_precision(y_train, y_train_preds, thresh))
print('Valid:%.3f'%calc_precision(y_valid, y_valid_preds, thresh))
print('Test:%.3f'%calc_precision(y_test, y_test_preds, thresh))

print('\nSpecificity')
print('Train:%.3f'%calc_specificity(y_train, y_train_preds, thresh))
print('Valid:%.3f'%calc_specificity(y_valid, y_valid_preds, thresh))
print('Test:%.3f'%calc_specificity(y_test, y_test_preds, thresh))

print('\nPrevalence')
print('Train:%.3f'%calc_prevalence(y_train))
print('Valid:%.3f'%calc_prevalence(y_valid))
print('Test:%.3f'%calc_prevalence(y_test))


plt.plot(fpr_train, tpr_train,'r-', label = 'Train AUC: %.2f'%auc_train)
plt.plot(fpr_valid, tpr_valid,'b-',label = 'Valid AUC: %.2f'%auc_valid)
plt.plot(fpr_test, tpr_test,'g-',label = 'Test AUC: %.2f'%auc_test)

plt.plot([0,1],[0,1],'-k')
plt.title('Best Model AUC-ROC')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix 

#Re-running predictions and see classification report on this grid object
clf_best_model = model.predict(X_test_vect) 

print('Logistic Classification Report\n\n', classification_report(y_test, clf_best_model, labels=[1,0]))
print('Confusion Matrix \n\n', confusion_matrix(y_test, clf_best_model, labels=[1,0]))

In [ ]:
import pickle
# save the model for later use
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))
 




In [ ]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open('model.sav', 'rb'))
result = model.score(X_test_vect, y_test)
print(result)